In [1]:
import pandas as pd
import re
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from collections import Counter

In [2]:

import pandas as pd
 
file_path = '../dataset/medical_texts/train.dat'
 
contents= []
with open(file_path, 'r') as f:
    for line in f:
        contents.append(line)

In [3]:
contents = [x.strip() for x in contents]

In [3]:
def removeTabsAtBegining(text):
    return re.sub(r'^\d+\t', '', text)

In [4]:
def extractClasses(text):
    return re.findall(r'^\d+', text)[0]

In [7]:
df['label'].value_counts()

label
0    4827
1     747
Name: count, dtype: int64

In [6]:
#df = pd.DataFrame({'text': contents, 'label': [1]*len(contents)})
df= pd.read_parquet("hf://datasets/ucirvine/sms_spam/plain_text/train-00000-of-00001.parquet")

c:\Users\Rohan\miniconda3\envs\env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
df.head(1)

,sms,label
0,"Go until jurong point, crazy.. Available only ...",0


In [8]:
df['label']= df['text'].apply(extractClasses)

KeyError: 'text'

In [ ]:
df['text']= df['text'].apply(removeTabsAtBegining)

In [9]:
df.head(2)

,sms,label
0,go until jurong point crazy available only in ...,0
1,ok lar joking wif u oni\n,0


In [9]:
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation (you might want to expand this)
    text = ''.join([c for c in text if c.isalnum() or c.isspace()])
    return text

In [10]:
df['sms'] = df['sms'].apply(preprocess_text)

In [14]:
class CustomTokenizer:
    def __init__(self):
        self.word_index = {}

    def fit_on_texts(self, texts):
        words = [word for text in texts for word in text.split()]
        word_counts = Counter(words)
        sorted_words = sorted(word_counts, key=word_counts.get, reverse=True)
        self.word_index = {word: i + 1 for i, word in enumerate(sorted_words)}

    def texts_to_sequences(self, texts):
        return [[self.word_index.get(word, 0) for word in text.split()] for text in texts]

In [11]:
CustomTokenizer = CustomTokenizer()
CustomTokenizer.fit_on_texts(df['sms'])

NameError: name 'CustomTokenizer' is not defined

In [12]:
max_words = 1000
max_length = 100

tokenizer = Tokenizer(num_words=max_words, oov_token='<OOV>')
tokenizer.fit_on_texts(df['sms'])

In [13]:
sequences = tokenizer.texts_to_sequences(df['sms'])
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')

In [14]:
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 16

In [13]:
df['label'] = df['label'].astype(int)

In [14]:
df['label'].nunique()

2

In [72]:
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length),
    LSTM(64, return_sequences=True),
    Dropout(0.5),
    LSTM(32, return_sequences=True),
    Dropout(0.5),
    Dense(8, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  
])

In [73]:
import tensorflow as tf
early_stop= tf.keras.callbacks.EarlyStopping(restore_best_weights=True, patience=3, verbose=1)

In [74]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

class_weights = compute_class_weight('balanced', classes=np.unique(df['label']), y=df['label'])
class_weight_dict = dict(enumerate(class_weights))

In [75]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [76]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 100, 16)           153360    
                                                                 
 lstm_9 (LSTM)               (None, 100, 64)           20736     
                                                                 
 dropout_12 (Dropout)        (None, 100, 64)           0         
                                                                 
 lstm_10 (LSTM)              (None, 100, 32)           12416     
                                                                 
 dropout_13 (Dropout)        (None, 100, 32)           0         
                                                                 
 dense_10 (Dense)            (None, 100, 8)            264       
                                                                 
 dropout_14 (Dropout)        (None, 100, 8)           

In [77]:
history = model.fit(padded_sequences, df['label'], epochs=50, batch_size=32, validation_split=0.2, verbose=1, callbacks=[early_stop], class_weight=class_weight_dict)

Epoch 1/50
140/140 [==============================] - 27s 99ms/step - loss: 0.5037 - accuracy: 0.6505 - val_loss: 0.2244 - val_accuracy: 0.9698
Epoch 2/50
140/140 [==============================] - 6s 43ms/step - loss: 0.2966 - accuracy: 0.9518 - val_loss: 0.2200 - val_accuracy: 0.9606
Epoch 3/50
140/140 [==============================] - 6s 45ms/step - loss: 0.2339 - accuracy: 0.9683 - val_loss: 0.1629 - val_accuracy: 0.9652
Epoch 4/50
140/140 [==============================] - 6s 42ms/step - loss: 0.2025 - accuracy: 0.9732 - val_loss: 0.1935 - val_accuracy: 0.9683
Epoch 5/50
140/140 [==============================] - 6s 42ms/step - loss: 0.1957 - accuracy: 0.9728 - val_loss: 0.1460 - val_accuracy: 0.9676
Epoch 6/50
140/140 [==============================] - 5s 39ms/step - loss: 0.2103 - accuracy: 0.9700 - val_loss: 0.1940 - val_accuracy: 0.9454
Epoch 7/50
140/140 [==============================] - 6s 40ms/step - loss: 0.1709 - accuracy: 0.9750 - val_loss: 0.1390 - val_accuracy: 0.970

In [78]:
def predict_medical(text):
    processed = preprocess_text(text)
    sequence = tokenizer.texts_to_sequences([processed])
    padded = pad_sequences(sequence, maxlen=max_length, padding='post')
    prediction = model.predict(padded)
    return prediction[0][0] 

In [80]:
predict_medical("""
    WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 10 hours only.	
""")

1/1 [==============================] - 0s 46ms/step


array([0.9036589], dtype=float32)

In [85]:
import tensorflow as tf

# Convert the model
converter= tf.lite.TFLiteConverter.from_keras_model(model)
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
converter._experimental_lower_tensor_list_ops = False
tflite_model= converter.convert()

with open('../exports/spamClassification.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\Rohan\AppData\Local\Temp\tmp3vufownm\assets


INFO:tensorflow:Assets written to: C:\Users\Rohan\AppData\Local\Temp\tmp3vufownm\assets


In [86]:
import numpy as np

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path="../exports/spamClassification.tflite")
interpreter.allocate_tensors()

# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Create a function to make predictions
def predict_spam_tflite(text):
    # Preprocess the text
    processed = preprocess_text(text)
    sequence = tokenizer.texts_to_sequences([processed])
    padded = pad_sequences(sequence, maxlen=max_length, padding='post')
    
    # Ensure the input is in the correct format (float32)
    input_data = padded.astype(np.float32)
    
    # Set the input tensor
    interpreter.set_tensor(input_details[0]['index'], input_data)
    
    # Run inference
    interpreter.invoke()
    
    # Get the output tensor
    output_data = interpreter.get_tensor(output_details[0]['index'])
    
    return output_data[0][0]  # For binary classification

# Test the function
test_text = "WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only."
result = predict_spam_tflite(test_text)
print(f"Spam probability: {result}")

Spam probability: [0.9036588]


In [15]:
import tensorflow as tf

vectorizer = tf.keras.layers.TextVectorization(
    max_tokens=max_words,  # Maximum vocabulary size
    output_mode='int',  # Output integer token sequences
    output_sequence_length=max_length  # Pad or truncate sequences to this length
)

vectorizer.adapt(df['sms'].values)

In [16]:
import pickle
with open('../deployment/utils/vectorizer_weights.pkl', 'wb') as f:
    pickle.dump(vectorizer.get_weights(), f)

In [28]:
print(vocab_size)

9585


In [16]:
import pickle

with open('../deployment/custom_tokenizer_scam.pickle', 'wb') as handle:
    pickle.dump(CustomTokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

PicklingError: Can't pickle <class '__main__.CustomTokenizer'>: it's not the same object as __main__.CustomTokenizer

In [25]:
from tokenizers import Tokenizer
from tokenizers.models import WordPiece
from tokenizers.trainers import WordPieceTrainer

# Initialize a new tokenizer
tokenizer = Tokenizer(WordPiece())
trainer = WordPieceTrainer(
    vocab_size=vocab_size,  # Adjust vocabulary size as needed
    min_frequency=2,  # Adjust minimum frequency as needed
    special_tokens=["[UNK]", "[PAD]", "[CLS]", "[SEP]"]
)

tokenizer.train_from_iterator((str(s) for s in df["sms"].values), trainer=trainer)

In [29]:
tokenizer.save("../deployment/utils/cusToken.json")

In [30]:
tf.__version__

'2.10.1'